In [2]:
import os
import pickle
def make_index(file):
    origins = []
    current_origin = 0

    original_position = file.tell()
    file.seek(0)

    while True:
        try:
            pickle.load(file)
            origins.append(current_origin)
            current_origin = file.tell()
        except EOFError:
            break
        except Exception as e:
            print(f"Exception: {e} in daqfile initializing index of file")

    #print(f"Make index {type(file)}")
    file.seek(original_position)

    return origins


filename = '20240827_FakeWBC_Vtrigger.npy'

try:
    file = open(filename, 'r+b')
except Exception as e:
    print(f"Exception: {e} in daqfile for playback")
    raise

origins = make_index(file)

In [3]:
#COUNT
if(origins is None):
    raise Exception("Daqfile: file has not been initialized")
len(origins)

16065

In [19]:
#GET
file.seek(origins[0])
packet = pickle.load(file)
print(len(packet))
print(type(packet))
print(packet.keys())
print(len(packet['event']['data']['adc_data']))
print(len(packet['event']['data']['adc_data'][0]))
temp = packet['event']['data']['adc_data'][500:1000]
print(len(temp))
print(len(temp[:, 22]))
# data = []
# for i in range(6):
#     file.seek(origins[i])
#     packet = pickle.load(file)
#     data.append(packet['event']['data']['adc_data'])

2
<class 'dict'>
dict_keys(['time', 'event'])
2500
36
500
500


: 

In [8]:
string1 = ["375 nm", "488 nm", "nc", "top right", "nc", "398 nm", "405 nm", "422 nm", "nc", "nc"]
string2 = {}
tempstr = []
for n in range(len(string1)):
    if string1[n].lower() != "nc":
        string2[f"{string1[n]}"] = n
    else:
        tempstr.append(n)

temp = list(string2.keys())
print(temp)
temp.sort()
temp.reverse()
print(temp)
sorted_dict = {i : string2[i] for i in temp}

print(sorted_dict)

print("\n\n\n")

sorted_dict["nc"] = tempstr
print(sorted_dict)
print(len(sorted_dict))
print(sorted_dict.keys())
print(sorted_dict["405 nm"])

['375 nm', '488 nm', 'top right', '398 nm', '405 nm', '422 nm']
['top right', '488 nm', '422 nm', '405 nm', '398 nm', '375 nm']
{'top right': 3, '488 nm': 1, '422 nm': 7, '405 nm': 6, '398 nm': 5, '375 nm': 0}




{'top right': 3, '488 nm': 1, '422 nm': 7, '405 nm': 6, '398 nm': 5, '375 nm': 0, 'nc': [2, 4, 8, 9]}
7
dict_keys(['top right', '488 nm', '422 nm', '405 nm', '398 nm', '375 nm', 'nc'])
6
hi


In [18]:
test = "['DAQ1', 'DAQ2', 'DAQ3']"

for i in ["'", "[", "]", ","]:
    test = test.replace(i, "")
# test = test.replace("'", "")
# test = test.replace("[", "")
# test = test.replace("]", "")
# test = test.replace(",", "")
print(test)
newtest = test.split(" ")
print(newtest)
print(type(newtest[0]))



DAQ1 DAQ2 DAQ3
['DAQ1', 'DAQ2', 'DAQ3']
<class 'str'>


In [28]:
lt = [('DAQ1.1', 'DAQ 0 Channel 0'), ('channel 1', 'DAQ 0 Channel 1'), ('channel 2', 'DAQ 0 Channel 2'), ('channel 3', 'DAQ 0 Channel 3'), ('channel 4', 'DAQ 0 Channel 4'), ('channel 5', 'DAQ 0 Channel 5'), ('channel 6', 'DAQ 0 Channel 6'), ('channel 7', 'DAQ 0 Channel 7'), ('channel 8', 'DAQ 0 Channel 8'), ('channel 9', 'DAQ 0 Channel 9'), ('channel 10', 'DAQ 0 Channel 10'), ('channel 11', 'DAQ 0 Channel 11'), ('channel 12', 'DAQ 1 Channel 0'), ('channel 13', 'DAQ 1 Channel 1'), ('channel 14', 'DAQ 1 Channel 2'), ('channel 15', 'DAQ 1 Channel 3'), ('channel 16', 'DAQ 1 Channel 4'), ('channel 17', 'DAQ 1 Channel 5'), ('channel 18', 'DAQ 1 Channel 6'), ('channel 19', 'DAQ 1 Channel 7'), ('channel 20', 'DAQ 1 Channel 8'), ('channel 21', 'DAQ 1 Channel 9'), ('channel 22', 'DAQ 1 Channel 10'), ('channel 23', 'DAQ 1 Channel 11'), ('channel 24', 'DAQ 2 Channel 0'), ('channel 25', 'DAQ 2 Channel 1'), ('channel 26', 'DAQ 2 Channel 2'), ('channel 27', 'DAQ 2 Channel 3'), ('channel 28', 'DAQ 2 Channel 4'), ('channel 29', 'DAQ 2 Channel 5'), ('channel 30', 'DAQ 2 Channel 6'), ('channel 31', 'DAQ 2 Channel 7'), ('channel 32', 'DAQ 2 Channel 8'), ('channel 33', 'DAQ 2 Channel 9'), ('channel 34', 'DAQ 2 Channel 10'), ('channel 35', 'DAQ 2 Channel 11'), ('channel 36', 'DAQ 3 Channel 0'), ('channel 37', 'DAQ 3 Channel 1'), ('channel 38', 'DAQ 3 Channel 2'), ('channel 39', 'DAQ 3 Channel 3'), ('channel 40', 'DAQ 3 Channel 4'), ('channel 41', 'DAQ 3 Channel 5'), ('channel 42', 'DAQ 3 Channel 6'), ('channel 43', 'DAQ 3 Channel 7'), ('channel 44', 'DAQ 3 Channel 8'), ('channel 45', 'DAQ 3 Channel 9'), ('channel 46', 'DAQ 3 Channel 10'), ('channel 47', 'DAQ 3 Channel 11')]
# lt = [('', 'DAQ 0 Channel 0'), ('channel 1', 'DAQ 0 Channel 1'), ('channel 2', 'DAQ 0 Channel 2'), ('channel 3', 'DAQ 0 Channel 3'), ('channel 4', 'DAQ 0 Channel 4'), ('channel 5', 'DAQ 0 Channel 5'), ('channel 6', 'DAQ 0 Channel 6'), ('channel 7', 'DAQ 0 Channel 7'), ('channel 8', 'DAQ 0 Channel 8'), ('channel 9', 'DAQ 0 Channel 9'), ('channel 10', 'DAQ 0 Channel 10'), ('channel 11', 'DAQ 0 Channel 11'), ('channel 12', 'DAQ 1 Channel 0'), ('channel 13', 'DAQ 1 Channel 1'), ('channel 14', 'DAQ 1 Channel 2'), ('channel 15', 'DAQ 1 Channel 3'), ('channel 16', 'DAQ 1 Channel 4'), ('channel 17', 'DAQ 1 Channel 5'), ('channel 18', 'DAQ 1 Channel 6'), ('channel 19', 'DAQ 1 Channel 7'), ('channel 20', 'DAQ 1 Channel 8'), ('channel 21', 'DAQ 1 Channel 9'), ('channel 22', 'DAQ 1 Channel 10'), ('channel 23', 'DAQ 1 Channel 11'), ('channel 24', 'DAQ 2 Channel 0'), ('channel 25', 'DAQ 2 Channel 1'), ('channel 26', 'DAQ 2 Channel 2'), ('channel 27', 'DAQ 2 Channel 3'), ('channel 28', 'DAQ 2 Channel 4'), ('channel 29', 'DAQ 2 Channel 5'), ('channel 30', 'DAQ 2 Channel 6'), ('channel 31', 'DAQ 2 Channel 7'), ('channel 32', 'DAQ 2 Channel 8'), ('channel 33', 'DAQ 2 Channel 9'), ('channel 34', 'DAQ 2 Channel 10'), ('channel 35', 'DAQ 2 Channel 11'), ('channel 36', 'DAQ 3 Channel 0'), ('channel 37', 'DAQ 3 Channel 1'), ('channel 38', 'DAQ 3 Channel 2'), ('channel 39', 'DAQ 3 Channel 3'), ('channel 40', 'DAQ 3 Channel 4'), ('channel 41', 'DAQ 3 Channel 5'), ('channel 42', 'DAQ 3 Channel 6'), ('channel 43', 'DAQ 3 Channel 7'), ('channel 44', 'DAQ 3 Channel 8'), ('channel 45', 'DAQ 3 Channel 9'), ('channel 46', 'DAQ 3 Channel 10'), ('channel 47', 'DAQ 3 Channel 11')]


for i in range(len(newtest)):
    print(f"i: {i}")
    for t in range(12):
        # if lt[(i*12) + t][0] == "":
        #     print("boo")
        #     continue
        if newtest[i] in lt[(i*12) + t][0]:
            print("yay")
            continue
        else:
            print(f"Error: {newtest[i]} != {lt[(i*12) + t][0]}")

i: 0
boo
Error: DAQ1 != channel 1
Error: DAQ1 != channel 2
Error: DAQ1 != channel 3
Error: DAQ1 != channel 4
Error: DAQ1 != channel 5
Error: DAQ1 != channel 6
Error: DAQ1 != channel 7
Error: DAQ1 != channel 8
Error: DAQ1 != channel 9
Error: DAQ1 != channel 10
Error: DAQ1 != channel 11
i: 1
Error: DAQ2 != channel 12
Error: DAQ2 != channel 13
Error: DAQ2 != channel 14
Error: DAQ2 != channel 15
Error: DAQ2 != channel 16
Error: DAQ2 != channel 17
Error: DAQ2 != channel 18
Error: DAQ2 != channel 19
Error: DAQ2 != channel 20
Error: DAQ2 != channel 21
Error: DAQ2 != channel 22
Error: DAQ2 != channel 23
i: 2
Error: DAQ3 != channel 24
Error: DAQ3 != channel 25
Error: DAQ3 != channel 26
Error: DAQ3 != channel 27
Error: DAQ3 != channel 28
Error: DAQ3 != channel 29
Error: DAQ3 != channel 30
Error: DAQ3 != channel 31
Error: DAQ3 != channel 32
Error: DAQ3 != channel 33
Error: DAQ3 != channel 34
Error: DAQ3 != channel 35


In [13]:
test = (0, '360nm')
print(type(test))
print(test[1][0].isdigit())

<class 'tuple'>
True


In [11]:
import datetime
months = [1,2,3,4,5,6,7,8,9,10]
month_labels = [
            # Generate a list of tuples (x_value, x_label)
            (m, datetime.date(2020, m, 1).strftime('%B'))
            for m in months
        ]
print(month_labels)

[(1, 'January'), (2, 'February'), (3, 'March'), (4, 'April'), (5, 'May'), (6, 'June'), (7, 'July'), (8, 'August'), (9, 'September'), (10, 'October')]


In [ ]:
# packet = file.get(record_n)
data1 = packet['event']['data']['adc_data']
print(data1)

In [11]:
temp = {}
print(temp.keys())
temp["check"] = [0,1,2,3,4,5]
temp["check1"] = [0,1,2,3,4,5]

n = temp.keys()
print(n)
if ("check" in n) and ("check1" in n):
    print("Achieved")


print(temp.keys())

print(temp)

dict_keys([])
dict_keys(['check', 'check1'])
Achieved
dict_keys(['check', 'check1'])
{'check': [0, 1, 2, 3, 4, 5], 'check1': [0, 1, 2, 3, 4, 5]}


Comparing with Valery Software

Numbers to match or get close to\
471476.88671875 \
717160.87109375 \
726431.96484375 \
742656.37890625 \
744974.15234375 \
1062509.11328125 \
Median gets closest so far


In [ ]:
import torch
import time
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(6):
    bigarr1 = torch.from_numpy(data[i].astype(float)).to(device)
    # bigarr1 = torch.from_numpy(savgol_filter(data[i], window_length=int(len(data[i][0])/10), polyorder=1, axis=1).astype(float)).to(device)

    # a1 = torch.std_mean(bigarr1, dim=0, keepdim=True)
    a1 = torch.std(bigarr1, dim=0, keepdim=True)
    b1 = torch.mean(bigarr1, dim=0).to(device)
    #Height
    e1 = (bigarr1.amax(dim=0) - bigarr1.amin(dim=0)).to(device)
    # e1 = (bigarr1.amax(dim=1).to(device) - bigarr1.amin(dim=1).to(device)) 
    # f1 = (bigarr1.lt(b1 - (a1*0.3)) * bigarr1).to(device)
    f1 = (bigarr1.lt(b1) * bigarr1).to(device)
    #WITDH
    g1 = f1.count_nonzero(dim=0).to(device)
    #AREA
    h1 = ((b1 * g1) - f1.sum(dim=0)).to(device)

    # print(a1)
    print("Height")
    print(e1.cpu().tolist())
    print("Width")
    print(g1.cpu().tolist())
    print("Area")
    print(h1.cpu().tolist())

In [ ]:
import torch
import time
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(6):
    bigarr1 = torch.from_numpy(data[i].astype(float)).to(device)
    # bigarr1 = torch.from_numpy(savgol_filter(data[i], window_length=int(data[i][0].__len__()/10), polyorder=1, axis=1).astype(float)).to(device)

    a1 = torch.std(bigarr1, dim=0, keepdim=True).to(device)
    b1 = torch.median(bigarr1, dim=0)
    e1 = (bigarr1.amax(dim=0) - bigarr1.amin(dim=0)).to(device)
    # e1 = (bigarr1.amax(dim=1).to(device) - bigarr1.amin(dim=1).to(device)) 
    f1 = (bigarr1.lt(b1[0]) * bigarr1).to(device)
    #WITDH
    g1 = f1.count_nonzero(dim=0).to(device)
    #AREA
    h1 = (((b1[0]) * g1) - f1.sum(dim=0)).to(device)

    # print(a1)
    print("Height")
    print(e1.cpu().tolist())
    print("Width")
    print(g1.cpu().tolist())
    print("Area")
    print(h1.cpu().tolist())

In [6]:
import time
i = 0
t = 0

while t < 10:
    time.sleep(1/120)
    i = (i + 1) % 8
    t+=1    
    print("120hz")
    
    if not (i % 2):
        print("60hz")
    if not (i % 4):
        print("30hz")
    if (i == 0):
        print("15hz")

120hz
120hz
60hz
120hz
120hz
60hz
30hz
120hz
120hz
60hz
120hz
120hz
60hz
30hz
15hz
120hz
120hz
60hz


2


In [3]:
a = [1,2,3,4,5,6,7,8,9]
del(a[-1])
del(a[-2])
print(a)

[1, 2, 3, 4, 5, 6, 8]


In [5]:
WRITE = False
WRITE = False if WRITE else True
print(WRITE)

In [4]:
import numpy as np

scale = np.logspace(start=0, stop=7, num = 20)
print(scale)

[1.00000000e+00 2.33572147e+00 5.45559478e+00 1.27427499e+01
 2.97635144e+01 6.95192796e+01 1.62377674e+02 3.79269019e+02
 8.85866790e+02 2.06913808e+03 4.83293024e+03 1.12883789e+04
 2.63665090e+04 6.15848211e+04 1.43844989e+05 3.35981829e+05
 7.84759970e+05 1.83298071e+06 4.28133240e+06 1.00000000e+07]


In [85]:
import pandas as pd
import numpy as np

f = open('datafile.csv', 'a')

tempH = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','a1','b1','c1','d1','e1','f1','g1','h1','i1','j1','k1','l1','m1','n1','o1','p1','q1','r1','s1','t1','u1','v1']
HEADER = []
HEADER += tempH
HEADER += tempH
HEADER += tempH

list1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
list2 = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
list3 = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
holdlist = list1+list2+list3
templist = np.asarray(holdlist)
print(templist)
# templist = np.asarray((list1,list2,list3))

# temp = dict(HEADER, templist)
# print(temp)

np.savetxt(f, np.transpose(HEADER), delimiter=',', fmt='%s')
np.savetxt(f, np.transpose(templist), delimiter=',', fmt='%d')

f.close()
# df = pd.DataFrame(templist)
# df = df.T
# print(df)

# df1 = pd.DataFrame(HEADER)
# df1.to_csv('datafile.csv', sep=',')
# df.to_csv('datafile.csv', sep=',', header=HEADER, mode='a')
# df.to_csv('datafile.csv', sep=',', mode='a')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [13]:
testlst = ['388nm', 'nc', 'nc', '488nm', 'QPD V']
testdict = {0 : '388nm', 1 : 'nc'}

# print(testdict.keys())
# print(testdict.values())
# print(list(testdict.values()).index('388nm'))

for i in testlst:
    if not i.replace(" ", "").isalpha():
        print(i)

# g = [i for i in testlst if not i.isalpha()]
# print(g)

388nm
488nm
['388nm', '488nm', 'QPD V']


In [21]:
import numpy as np

q = [1,2,3,4,5,6,7,8]
# np.array(1,2,3,4,5, dtype=np.float64)
np.array(q, dtype=np.float64)

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [23]:
import os
import sys

def funtime(x,y):
    return x + y

In [35]:
import numpy as np

tstlst = np.array(
    [[1,2,3],
     [3,4,5],
     [5,6,7],
     [7,8,9]]

)

print(tstlst[::])
# tstlst = tstlst + 1
# tstlst = [x+1 for x in tstlst]
# print(tstlst)

[[1 2 3]
 [3 4 5]
 [5 6 7]
 [7 8 9]]


: 

In [9]:
import os, os.path

# simple version for working with CWD
print (len([name for name in os.listdir('.') if os.path.isfile(name)]))
temp = [name for name in os.listdir('.') if os.path.isfile(name)]
print(temp)
temp = [name for name in os.listdir('./CMAKE')]
print(temp)
newtemp = []

for n in temp:
    if '.txt' in n:
        newtemp.append(n)

print(newtemp)

11
['.gitignore', 'databasetest.ipynb', 'dbout1.txt', 'mathing.ipynb', 'out2.txt', 'processingtesting.ipynb', 'processtest.py', 'protobuf.ipynb', 'test.bat', 'test.ini', 'testing.ipynb']
['CMakeLists(det).txt']
['CMakeLists(det).txt']


In [7]:
import numpy as np
import math
binsLogy = np.logspace(start=math.log10(50000), stop=math.log10(1000000), num=1024, base=10.0)
print(binsLogy)

print(math.log10(1000000))

[  50000.           50146.63357179   50293.69717167 ...  994160.35829167
  997075.90397706 1000000.        ]
6.0


In [1]:
binsLogx = np.linspace(start=5000, stop=1000000, num=1024)
print(binsLogx[1023])

NameError: name 'np' is not defined

In [8]:
while True:
    for i in range(10):
        if i == 5:
            break
    print("i: ", i)
    for i in range(10):
        print("i: ", i)
    
    break

SyntaxError: invalid syntax (1430120190.py, line 4)

In [46]:
import numpy as np

tst = [0,1,2,3,4,5,6,7,8,9]
tst = (np.array(tst) * 3).tolist()
print(tst)

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]


In [55]:
x = []
x.append([1,2,3,4])
print(type(x[0]))
print(len(x))
x.count

<class 'list'>
1


In [98]:
class test:
    def __init__(self):    
        self.dict = {'x': 1, 'y': 2, 'z': 3}
        self.lst = [1,2,3,4,5,6,7,8,9,10]
        self.x = 0
        
    def change_attr(self, values):
        dct = {}
        for i in range(len(values)):
            dct[list(self.dict.keys())[i]] = values[i]
            
        setattr(self, 'dict', dct)
        
tst = test()
print(tst.dict)
tst.change_attr([0,0,0])
print(tst.dict)

False


In [ ]:
import json
x = json.loads('{"depth": 1.05, "sample": 0.0, "vaccum": 0.0}')

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [31]:
from multiprocessing import Manager, Value
import multiprocessing
import ctypes
mp_context = multiprocessing.get_context('spawn')
mgr = mp_context.Manager()
tst = mgr.list()
for i in range(2):
    tst.append({'x': i, 'y': i*2, 'z': i*3})
print(tst)
temp = tst[0]
temp['x'] = 100
tst[0] = temp
print(tst)

[{'x': 0, 'y': 0, 'z': 0}, {'x': 1, 'y': 2, 'z': 3}]
[{'x': 100, 'y': 0, 'z': 0}, {'x': 1, 'y': 2, 'z': 3}]


In [37]:
dict1 = {'x': 1, 'y': 2, 'z': 3}
dict2 = {'a': 4, 'b': 5, 'c': 6}
dict3 = dict1 | dict2
print(dict3)
for i in range(len(dict3)):
    print(list(dict3.values())[i])
dict1.__len__()


{'x': 1, 'y': 2, 'z': 3, 'a': 4, 'b': 5, 'c': 6}
1
2
3
4
5
6


3

In [38]:
H = None
if not H:
    print("H is None")

H is None


In [13]:
import numpy as np
arr = [1,2,3,4,5,6,7,8,9,10]
arr1 = arr[0:5]
arr2 = arr[5:]
arr3 = np.array([arr1, arr2])
print(arr3)
print(type(arr3))
arr4 = arr3[:, [2,3]].tolist()
print(arr4)
print(type(arr4))

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]
<class 'numpy.ndarray'>
[[3, 4], [8, 9]]
<class 'list'>


: 